In [ ]:
%%javascript

import Root from './treezjs/src/root/root.js';
import TableSourceType from '../notebooks/treezjs/src/data/table/tableSourceType.js';
import Direction from './treezjs/src/result/axis/direction.js';

window.createModel = function(){

    var root = new Root();

    //models---------------------------------------------------------
    var models = root.createModels();
       
    var path = models.createPath();
    path.createDirectoryPathVariable('workingDirectory', 'D:/Sensitivity/trunk/forecast');
    path.createDirectoryPathVariable('forecastPath', 'D:/forecast4');
    path.createDirectoryPathVariable('projectPath', '{$forecastPath$}/trunk/databases/Demo');
    path.createFilePathVariable('inputFilePath', '{$projectPath$}/demo_default_project.xml');
       
    var genericInput = models.createGenericInput();   
    genericInput.createIntegerVariable('HowEasy',1);
    
    var databaseModifier = models.createDatabaseModifier();
    databaseModifier.targetFilePath =  '{$projectPath$}/demo.sqlite';
    databaseModifier.query = "UPDATE 'ScenarioData_AddTo_Base' SET 'Y2000' = {$HowEasy$}";
    
    var inputFileGenerator = models.createInputFileGenerator();
    inputFileGenerator.templatePath = '{$projectPath$}/template.xml';        
    inputFileGenerator.inputFilePath = '{$inputFilePath$}';
    inputFileGenerator.valueExpression='<value>';   
   
    var executable = models.createExecutable();
    executable.executablePath = '{$forecastPath$}/trunk/source/Model/bin/Debug/Model.exe';    
    executable.inputArguments = '-path';
    executable.inputPath = '{$inputFilePath$}';     

    var tableImport = models.createTableImport();
    tableImport.resultTableModelPath = 'root.results.data.table';
    tableImport.type = TableSourceType.sqLite;
    tableImport.tableName = 'Calculated_ModelParameter_TimeSeries_Result';
    tableImport.filePath = '{$projectPath$}/demo.sqlite';
    
    //var sqLiteAppender = models.createSqLiteAppender();
    //sqLiteAppender.sourceFilePath = projectPath + '/demo_results.sqlite';
    //sqLiteAppender.targetFilePath = projectPath + '/demo_study_results.sqlite';
    
    //studies------------------------------------------------------------
    var studies = root.createStudies();

    //sweep
    var sensitivity = studies.createSensitivity();
    sensitivity.id = 'mystudyId';
    sensitivity.description = 'myStudyDescription';
    sensitivity.controlledModelPath = 'root.models';
    sensitivity.sourceModelPath = 'root.models.genericInput';   
    
    var pythonExport = sensitivity.createPythonExport();

    //results------------------------------------------------------------
    var results = root.createResults();    
    var data = results.createData();    
    
    /*
    var sensitivityProbe = data.createSensitivityProbe();
    sensitivityProbe.sensitivityOutputPath = 'root.results.data.sensitivityOutput';
    sensitivityProbe.firstProbeTablePath = sweepProbe.sensitivityOutputPath + '.output_1.executable.tableImportOutput';
    sensitivityProbe.columnIndex = 11;
    sensitivityProbe.rowIndex = 1;

    */
    
    var page = results.createPage();
    var graph = page.createGraph();
    graph.border.isHidden=true;

    var xAxis = graph.createAxis('x');
    xAxis.data.label='x';    

    var yAxis = graph.createAxis('y');
    yAxis.data.label='y';
    yAxis.data.direction = Direction.vertical;

    var xy = graph.createXy();
    xy.data.xData = 'root.results.data.sensitivityProbe.sensitivityProbeTable.columns.x';
    xy.data.yData = 'root.results.data.sensitivityProbe.sensitivityProbeTable.columns.y';

    xy.data.xAxis = 'root.results.page.graph.x';
    xy.data.yAxis = 'root.results.page.graph.y';

    return root;            
};
